<a href="https://colab.research.google.com/github/DJCordhose/practical-llm/blob/main/Phi-3.5-vision_L4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Maßblatt auf Phi-3.5-vision

* https://www.medi.de/fachhandel/wissen/produktwissen/phlebologie-lymphologie/vorlagen-vermessung/
* https://huggingface.co/microsoft/Phi-3.5-vision-instruct
* https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/discover-the-new-multi-lingual-high-quality-phi-3-5-slms/ba-p/4225280


In [1]:
%%time

!pip install --upgrade -q transformers accelerate flash_attn torch bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 69.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu121 requires torch==2.3.1, but you have torch 2.4.0 which is incompatible.
torchvision 0.18.1+cu121 requires torch==2.3.1, but 

In [2]:
!pip install Pillow torchvision Requests -q --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 99.9 MB/s eta 0:00:00


In [3]:
import transformers
transformers.__version__


'4.44.2'

In [4]:
import accelerate
accelerate.__version__

'0.33.0'

In [5]:
import flash_attn
flash_attn.__version__

'2.6.3'

In [6]:
import torch
torch.__version__

'2.4.0+cu121'

In [7]:
import bitsandbytes
bitsandbytes.__version__

'0.43.3'

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
from IPython.display import Markdown

In [13]:
!nvidia-smi

Fri Aug 23 08:15:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   45C    P8              16W /  72W |      4MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [29]:
from PIL import Image
import requests
from transformers import AutoModelForCausalLM
from transformers import AutoProcessor

model_id = "microsoft/Phi-3.5-vision-instruct"

# Note: set _attn_implementation='eager' if you don't have flash_attn installed
model = AutoModelForCausalLM.from_pretrained(
  model_id,
  device_map="cuda",
  trust_remote_code=True,
  torch_dtype="auto",
  _attn_implementation='flash_attention_2'
)

# for best performance, use num_crops=4 for multi-frame, num_crops=16 for single-frame.
processor = AutoProcessor.from_pretrained(model_id,
  trust_remote_code=True,
  num_crops=16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [61]:
ocr = '''
60 cG 66
49 CF 57
42 cE stehend 46
41 CE 44
36,5 CD 38
35 37 cC 42
29 CB1 33
21 CB 24
34 CY
23,5 cA 24,5
'''

In [42]:
# image = Image.open("/content/Anmerkung 2024-08-23 102153.png")
# image = Image.open("/content/Anmerkung 2024-08-23 103947.png")
image = Image.open("/content/Umfangmaße_rechts.png")

In [69]:
images = [image]
# mass = "cE"
mass = "cY"
# zug_haut = "Zugmaß"
zug_haut = "Hautmaß"
messages = [
#     {"role": "system", "content": '''
# Es geht um ein Maßblatt für Kompressionsstrümpfe.
# In diesem Blatt stehen unterschiedliche Maße mit durchgezogenen Linien getrennt.
# Oben steht welche Spalte das Zugmaß und welche Spalte das Hautmaß sind.
# Suche ein Maß aus dem Blatt heraus. Gib nur dieses Maß und eine Erklärung wo das Maß auf dem Blatt ist zurück.
# '''},
    # {"role": "user", "content": "<|image_1|>\n. Was ist das Hautmaß für cT? Wo findest du den Wert auf dem Maßblatt?"},
    # {"role": "user", "content": "<|image_1|>\n. Was ist das Zugmaß für cE für das rechte Bein? Wo findest du den Wert auf dem Maßblatt?"},
    {"role": "user", "content":
# '''
# <|image_1|>
# Was ist das Zugmaß für cG?
# '''
# '''
# <|image_1|>
# Was ist das Hautmaß für cE (nicht stehend)?
# '''
# '''
# <|image_1|>
# Was ist das Zugmaß für cE (nicht stehend)?
# '''
# '''
# <|image_1|>
# Lies alle Maße aus dem Maßblatt aus und gib sie als ein JSON mit Name/Wert zurück.
# Der Name für ein Maß steht zwischen den beiden Werten für Zugmaß und Hautmaß.
# '''
f'''
Es geht um ein Maßblatt für Kompressionsstrümpfe.
In diesem Blatt stehen unterschiedliche Maße mit durchgezogenen horizontalen Linien getrennt.
Der Name für ein Maß steht zwischen den beiden Werten für Zugmaß und Hautmaß.
Das Zugmaß steht links, das Hautmaß rechts vom Namen.

Suche ein Maß aus dem Blatt heraus. Gib dieses Maß und wo das Maß auf dem Blatt ist.
Wenn es innerhalb horizontalen Linien mehr als einen Wert gibt, liefere alle Werte zurück.

<|image_1|>
Was ist das {zug_haut} für {mass}?

Zur Ergänzung hier der OCRte Text aus dem Maßblatt:
{ocr}
'''

},
]

prompt = processor.tokenizer.apply_chat_template(
  messages,
  tokenize=False,
  add_generation_prompt=True
)

inputs = processor(prompt, images, return_tensors="pt").to("cuda:0")


generation_args = {
    "max_new_tokens": 1000,
    "temperature": 0.0,
    "do_sample": False,
}

generate_ids = model.generate(**inputs,
  eos_token_id=processor.tokenizer.eos_token_id,
  **generation_args
)

# remove input tokens
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids,
  skip_special_tokens=True,
  clean_up_tokenization_spaces=False)[0]

print(response)

Das Hautmaß für cY ist 34.


In [40]:
messages = [
    {"role": "system", "content": "Es geht um ein Maßblatt für Kompressionsstrümpfe. Suche ein Maß aus dem Blatt heraus. Gib nur dieses Maß und eine Erklärung wo das Maß auf dem Blatt ist zurück."},
    # {"role": "user", "content": "<|image_1|>\n. Was ist das Hautmaß für cT? Wo findest du den Wert auf dem Maßblatt?"},
    # {"role": "user", "content": "<|image_1|>\n. Was ist das Zugmaß für cE für das rechte Bein? Wo findest du den Wert auf dem Maßblatt?"},
    {"role": "user", "content": "<|image_1|>\n. Was ist das Zugmaß für cG für das rechte Bein?"},
    {"role": "assistant", "content": response},
    {"role": "user", "content": "Du gibst das Hautmaß zurück, wir wollten aber das Zugmaß. Wieso ist das so?"}
]

prompt = processor.tokenizer.apply_chat_template(
  messages,
  tokenize=False,
  add_generation_prompt=True
)

inputs = processor(prompt, images, return_tensors="pt").to("cuda:0")


generation_args = {
    "max_new_tokens": 1000,
    "temperature": 0.0,
    "do_sample": False,
}

generate_ids = model.generate(**inputs,
  eos_token_id=processor.tokenizer.eos_token_id,
  **generation_args
)

# remove input tokens
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids,
  skip_special_tokens=True,
  clean_up_tokenization_spaces=False)[0]

print(response)



Es scheint ein Fehler in der Anfrage zu sein. Das Hautmaß ist nicht dasselbe wie das Zugmaß. Das Hautmaß ist ein Maß für die Dicke der Haut, während das Zugmaß ein Maß für die Länge der Beinmuskulatur ist.


In [70]:
!nvidia-smi

Fri Aug 23 09:31:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   71C    P0              32W /  72W |  17281MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--